In [1]:
import pandas as pd

# First merge -- Combine SeatGeek and Bandsintown Artist Information

The Bandsintown artist info is already a subset of the SeatGeek artist info, since I used the list of bands from SeatGeek to pull artist information from Bandsintown. Therefore, I'll begin by merging the SeatGeek information into the Bandsintown artist information. Furthermore, I know these match on artist name because that is what I used to pull information from Bandsintown.

In [2]:
BIT_artist_df = pd.read_csv("BIT_artist_info.csv")
BIT_artist_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [3]:
SG_artist_df = pd.read_csv("sg_bands.csv")
SG_artist_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
BIT_artist_df.head()

,BIT_artist_id,BIT_artist_name,BIT_tracker_count
0,157,Taylor Swift,5703387
1,190899,Ed Sheeran,4293012
2,289995,Charli XCX,643712
3,46025,"""Honda Civic Tour""",899
4,2639,Eric Clapton,1388530


In [5]:
SG_artist_df.head()

,genre,id,name,score
0,Pop,35,Taylor Swift,0.88
1,Pop,13546,Ed Sheeran,0.87
2,Pop,16709,Charli XCX,0.87
3,Rock,10379,Honda Civic Tour,0.87
4,Classic Rock,696,Eric Clapton,0.86


In [6]:
print(BIT_artist_df.shape)
print(SG_artist_df.shape)

(38782, 3)
(43329, 4)


In [7]:
all_artist_df = pd.merge(BIT_artist_df, SG_artist_df, how='left', left_on='BIT_artist_name',
                         right_on='name', sort=False)

We no longer need the SeatGeek artist ID or name, and I'll rename the SG artist score column for clarity. Furthermore, we need to remove duplicate artists prior to merging.

In [8]:
all_artist_df.drop(['name', 'id'], axis=1, inplace=True)
all_artist_df.rename(columns={'score':'SG_artist_score'}, inplace=True)
all_artist_df = all_artist_df.drop_duplicates(['BIT_artist_id'])
print(all_artist_df.shape)
all_artist_df.head()

(38223, 5)


,BIT_artist_id,BIT_artist_name,BIT_tracker_count,genre,SG_artist_score
0,157,Taylor Swift,5703387,Pop,0.88
1,190899,Ed Sheeran,4293012,Pop,0.87
2,289995,Charli XCX,643712,Pop,0.87
3,46025,"""Honda Civic Tour""",899,NaN,NaN
4,2639,Eric Clapton,1388530,Classic Rock,0.86


In [9]:
all_artist_df.isna().sum()

BIT_artist_id           0
BIT_artist_name         0
BIT_tracker_count       0
genre                4427
SG_artist_score      4427
dtype: int64

Looks like some of these aren't really artists, including "Honda Civic Tour" -- let's drop them. After dropping those, I can convert the artist ID to an integer to prepare for the merge.

In [10]:
all_artist_df = all_artist_df.dropna()
all_artist_df.shape

(33796, 5)

In [11]:
all_artist_df = all_artist_df.astype({"BIT_artist_id": int})

# Second Merge -- Combine Artist and Gig Information

Next I'll merge the gig information into the combined artist information. The Bandsintown artist ID is present in each file and is used to connect gig information to the associated artist.

In [12]:
gig_df = pd.read_csv("all_gigs.csv")
gig_df.drop(['Unnamed: 0'], axis=1, inplace=True)
gig_df.head()

,BIT_artist_id,BIT_event_date,BIT_event_id,BIT_venue_city,BIT_venue_country,BIT_venue_name,BIT_venue_state
0,157,2013-04-10T19:00:00,5727347,Miami,United States,AmericanAirlines Arena,FL
1,157,2013-04-11T19:00:00,5821019,Orlando,United States,Amway Center,FL
2,157,2013-04-12T19:00:00,5821020,Orlando,United States,Amway Center,FL
3,157,2013-04-18T19:00:00,5691280,Atlanta,United States,Philips Arena,GA
4,157,2013-04-19T19:00:00,5691281,Atlanta,United States,Philips Arena,GA


In [13]:
gigs_artists_df = pd.merge(gig_df, all_artist_df, how='left', validate="m:1")
print(gigs_artists_df.shape)
gigs_artists_df.head()

(2747682, 11)


,BIT_artist_id,BIT_event_date,BIT_event_id,BIT_venue_city,BIT_venue_country,BIT_venue_name,BIT_venue_state,BIT_artist_name,BIT_tracker_count,genre,SG_artist_score
0,157,2013-04-10T19:00:00,5727347,Miami,United States,AmericanAirlines Arena,FL,Taylor Swift,5703387,Pop,0.88
1,157,2013-04-11T19:00:00,5821019,Orlando,United States,Amway Center,FL,Taylor Swift,5703387,Pop,0.88
2,157,2013-04-12T19:00:00,5821020,Orlando,United States,Amway Center,FL,Taylor Swift,5703387,Pop,0.88
3,157,2013-04-18T19:00:00,5691280,Atlanta,United States,Philips Arena,GA,Taylor Swift,5703387,Pop,0.88
4,157,2013-04-19T19:00:00,5691281,Atlanta,United States,Philips Arena,GA,Taylor Swift,5703387,Pop,0.88


# Third Merge -- Add Venue Scores to the Combined Artist and Gig Information

Finally we'll load in the venue data from SeatGeek and merge the venue popularity scores into each gig. The combined artist and gig information file is a gig-level file, but does not have the SeatGeek venue ID for each gig yet, so I need to merge on each venue's name and state.

In [14]:
SG_venue_df = pd.read_csv("sg_venues.csv")
SG_venue_df.drop(['Unnamed: 0'], axis=1, inplace=True)
SG_venue_df.head()

,sg_venue_id,venue_city,venue_country,venue_name,venue_score,venue_state
0,4965,Arlington,US,AT&T Stadium,0.975141,TX
1,58670,Santa Clara,US,Levi's Stadium,0.957688,CA
2,1132,San Francisco,US,Candlestick Park,0.950707,CA
3,1587,East Rutherford,US,MetLife Stadium,0.948735,NJ
4,31,Chicago,US,Soldier Field,0.940013,IL


In [15]:
gigs_artists_venues_df = pd.merge(gigs_artists_df, SG_venue_df, how='left', 
                                  left_on=['BIT_venue_name', 'BIT_venue_state'],
                                  right_on=['venue_name', 'venue_state'],
                                  validate="m:m")
print(gigs_artists_venues_df.shape)
gigs_artists_venues_df.head()

(2963474, 17)


,BIT_artist_id,BIT_event_date,BIT_event_id,BIT_venue_city,BIT_venue_country,BIT_venue_name,BIT_venue_state,BIT_artist_name,BIT_tracker_count,genre,SG_artist_score,sg_venue_id,venue_city,venue_country,venue_name,venue_score,venue_state
0,157,2013-04-10T19:00:00,5727347,Miami,United States,AmericanAirlines Arena,FL,Taylor Swift,5703387,Pop,0.88,183.0,Miami,US,AmericanAirlines Arena,0.841048,FL
1,157,2013-04-11T19:00:00,5821019,Orlando,United States,Amway Center,FL,Taylor Swift,5703387,Pop,0.88,2652.0,Orlando,US,Amway Center,0.752291,FL
2,157,2013-04-12T19:00:00,5821020,Orlando,United States,Amway Center,FL,Taylor Swift,5703387,Pop,0.88,2652.0,Orlando,US,Amway Center,0.752291,FL
3,157,2013-04-18T19:00:00,5691280,Atlanta,United States,Philips Arena,GA,Taylor Swift,5703387,Pop,0.88,NaN,NaN,NaN,NaN,NaN,NaN
4,157,2013-04-19T19:00:00,5691281,Atlanta,United States,Philips Arena,GA,Taylor Swift,5703387,Pop,0.88,NaN,NaN,NaN,NaN,NaN,NaN


# Cleaning the Final Merged File

First I'm going to drop any rows with missing values since that indicates a failed match, then I'll check to see if any non-US shows remain in the merged file, because I'm modeling shows in US cities only.

In [16]:
gigs_artists_venues_df.isna().sum()

BIT_artist_id              0
BIT_event_date             0
BIT_event_id               0
BIT_venue_city           138
BIT_venue_country        157
BIT_venue_name             5
BIT_venue_state       883124
BIT_artist_name       369374
BIT_tracker_count     369374
genre                 369374
SG_artist_score       369374
sg_venue_id          1795967
venue_city           1795978
venue_country        1796307
venue_name           1795967
venue_score          1795967
venue_state          1796012
dtype: int64

In [17]:
gigs_artists_venues_df = gigs_artists_venues_df.dropna()
gigs_artists_venues_df.shape

(1021395, 17)

It looks like we're already set on the US-only shows based on my merge specs and inputs.

In [18]:
gigs_artists_venues_df['BIT_venue_country'].value_counts()

United States               1018584
United States of America       1968
USA                             798
US                               35
Usa                               4
Us                                4
us                                1
UNITED STATES                     1
Name: BIT_venue_country, dtype: int64

Note that there are a lot of venue API calls that failed (these have a venue score of zero) due to inconsistencies in venue names, which could be cleaned up -- however, while I'm familiar enough with Chicago venues to do this in a reasonable amount of time, this would be very time-consuming across other locations.

That said, not all of the zero-score venues are incorrect -- below is a list of all zero-score venues in our database (which I'll then drop). Some are clearly correct, such as private shows and living room/house shows, and others, like Coachella and Lincoln Hall, just didn't show up in the SeatGeek venue database due to spelling variations. This is a target for future cleaning, but will be time-consuming.

In [19]:
gigs_artists_venues_df.loc[(gigs_artists_venues_df['venue_score'] == 0)]['venue_name'].value_counts()

Private                                       11171
House Show                                     5129
First Baptist Church                           3840
Living Room Show                               3392
First Avenue                                   2416
Iridium                                        2312
The Social                                     2256
Black Sheep                                    2032
The Catalyst                                   1968
The Fillmore Charlotte                         1863
The Loft                                       1802
Beacon Theatre                                 1744
Grand Ole Opry                                 1725
The Basement                                   1681
Birdland                                       1610
Cat's Cradle                                   1597
Club Congress                                  1580
Coachella                                      1580
The Fillmore                                   1562
3rd & Lindsl

In [20]:
gigs_artists_venues_df = gigs_artists_venues_df.loc[(gigs_artists_venues_df['venue_score'] > 0)]
gigs_artists_venues_df.shape

(518342, 17)

# Saving the Final Combined File

In [21]:
gigs_artists_venues_df.to_csv('merged_data.csv')